# Exercício 1

Qual é o caminho mais rápido para ir de 1 a N, sabendo que as únicas duas ações possíveis são:
 - andar de i para i + 1, com duração de 1 minuto (para todo i de 1 a N - 1)
 - mágica de i para 2*i, com duração de 2 minutos (para todo i de 1 a N/2)

Implemente as funções da classe abaixo que define a interface desse problema para utilização com algoritmos de busca vistos em sala de aula. Em sequência, escolha e implemente um dos algoritmos de busca explicados em sala que resolva o problema. Faça essa implementação na função *solution*, que tem como input um objeto criado utilizando a classe implementada *TransportationProblem*.

In [ ]:
from collections import deque
from queue import PriorityQueue

class TransportationProblem:
    def __init__(self, N):
        self.N = N

    def startState(self):
        # Estado inicial é 1
        return 1

    def isEnd(self, state):
        # Verifica se o estado é o estado final
        return state == self.N

    def succAndCost(self, state):
        # Retorna uma lista de tuplas com 3 informações:
        # (ação, novo estado, custo)

        successors = []

        # Adiciona ação 'walk' para estados de 1 a N-1
        if state < self.N:
            successors.append(('walk', state + 1, 1))

        # Adiciona ação 'magic' para estados de 1 a N/2
        if state * 2 <= self.N:
            successors.append(('magic', state * 2, 2))

        return successors

In [ ]:
def solution(problem):
    # Utilizando uma fila de prioridade para a fronteira
    frontier = PriorityQueue()
    frontier.put((0, problem.startState()))  # Tupla (custo, estado)

    came_from = {}  # Dicionário para armazenar de onde veio cada estado
    cost_so_far = {problem.startState(): 0}  # Dicionário para armazenar custo até o momento para cada estado

    while not frontier.empty():
        current_cost, current_state = frontier.get()

        if problem.isEnd(current_state):
            # Reconstruir o caminho a partir do estado final
            path = []
            while current_state in came_from:
                action, next_state, cost = came_from[current_state]
                path.append((action, current_state, cost))
                current_state = next_state
            path.reverse()
            return path

        for action, next_state, cost in problem.succAndCost(current_state):
            new_cost = cost_so_far[current_state] + cost

            if next_state not in cost_so_far or new_cost < cost_so_far[next_state]:
                cost_so_far[next_state] = new_cost
                came_from[next_state] = (action, current_state, cost)
                frontier.put((new_cost, next_state))  # Adiciona na fila de prioridade com o novo custo

# Exemplos de uso
N = 2
p = TransportationProblem(N)
print(*solution(p), sep='\n')

('walk', 2, 1)


Exemplos:


> N = 2 \
> p = TransportationProblem(N) \
> print(*solution(p), sep='\n') \

('walk', 2, 1)

=====================================

> N = 3 \
> p = TransportationProblem(N) \
> print(*solution(p), sep='\n') \

('walk', 2, 1) \
('walk', 3, 1)

=====================================

> N = 10 \
> p = TransportationProblem(N) \
> print(*solution(p), sep='\n') \

('walk', 2, 1) \
('magic', 4, 2) \
('walk', 5, 1) \
('magic', 10, 2) \

ou

('walk', 2, 1) \
('walk', 3, 1) \
('walk', 4, 1) \
('walk', 5, 1) \
('magic', 10, 2) \

(**nesse caso, os caminhos apresentam o mesmo custo total, logo não é necessário retornar as duas soluções, basta uma delas**)

=====================================

> N = 12 \
> p = TransportationProblem(N) \
> print(*solution(p), sep='\n') \

('walk', 2, 1) \
('walk', 3, 1) \
('magic', 6, 2) \
('magic', 12, 2) \

In [ ]:
# Exemplos
N = 2
p = TransportationProblem(N)
print(*solution(p), sep='\n')

N = 3
p = TransportationProblem(N)
print(*solution(p), sep='\n')

N = 10
p = TransportationProblem(N)
print(*solution(p), sep='\n')

N = 12
p = TransportationProblem(N)
print(*solution(p), sep='\n')

('walk', 2, 1)
('walk', 2, 1)
('walk', 3, 1)
('walk', 2, 1)
('magic', 4, 2)
('walk', 5, 1)
('magic', 10, 2)
('walk', 2, 1)
('walk', 3, 1)
('magic', 6, 2)
('magic', 12, 2)


Implemente uma soução para o problema com o algoritmo A* e uma solução com Dynamic Programing. Compare em gráfico o tempo de execução dos dois algoritmos e da sua solução (caso ela tenha sido diferente) para N variando de 10 a 500 (caso esteja demorando muito para Ns maiores, vá até o maior valor de N que conseguir).

In [ ]:
import time
import matplotlib.pyplot as plt

def heuristica(state, goal):
    # Heurística simples: distância até o estado final
    if type(goal) == int:
        goal = [goal]  # Convert integer to a list
    if type(state) == int:
        state = [state]  # Convert integer to a list
    return abs(goal[0] - state[0])

def solutionAstar(problem):
    frontier = PriorityQueue()
    frontier.put((0, problem.startState(), None, 0))  # Tupla (prioridade, estado, ação, custo)
    came_from = set()
    cost_so_far = {problem.startState(): 0}

    while not frontier.empty():
        _, current_state, action, _ = frontier.get()

        if problem.isEnd(current_state):
            # Reconstruir o caminho a partir do estado final usando deque
            path = deque()
            while action is not None:
                path.appendleft((action, current_state, 0))
                action, current_state, _ = came_from.pop()
            return path

        for next_action, next_state, cost in problem.succAndCost(current_state):
            new_cost = cost_so_far[current_state] + cost
            heuristic_cost = heuristica(next_state, problem.N)
            total_cost = new_cost + heuristic_cost

            if next_state not in cost_so_far or new_cost < cost_so_far[next_state]:
                cost_so_far[next_state] = new_cost
                frontier.put((total_cost, next_state, next_action, new_cost))
                came_from.add((next_action, next_state, current_state))


def solutionDP(problem):
    # Inicialização da tabela DP
    dp_table = {i: (float('inf'), None) for i in range(1, problem.N + 1)}
    dp_table[problem.N] = (0, None)

    # Preenchimento da tabela DP
    for state in range(problem.N - 1, 0, -1):
        walk_cost = dp_table[state + 1][0] + 1
        dp_table[state] = min(dp_table[state], (walk_cost, 'walk'))

        if state * 2 <= problem.N:
            magic_cost = dp_table[state * 2][0] + 2
            dp_table[state] = min(dp_table[state], (magic_cost, 'magic'))

    # Reconstrução do caminho a partir da tabela DP
    path = []
    current_state = 1
    while current_state is not None:
      _, action = dp_table[current_state]
      if action is not None:
          path.append((action, current_state, 0))  # O terceiro elemento é irrelevante aqui
          if action == 'walk':
              current_state += 1
          elif action == 'magic':
              current_state *= 2
      else:
          current_state = None

    path.reverse()
    return path

# Testando a implementação inicial
N = 10
p = TransportationProblem(N)
print("A*:", *solutionAstar(p), sep='\n')
print("DP:", *solutionDP(p), sep='\n')


KeyError: ignored

**Dica para medir o tempo de execução de uma função:**

Mais informações no [link da documentação](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit).

In [ ]:
def dummyFunc(N = 10):
    s = sum(range(N))

t = %timeit -q -o dummyFunc()
#t = %timeit -n 1 -q -o dummyFunc() # para o caso do comando acima estar demorando muito

t.average

3.194785962857054e-07

In [ ]:
import time

def compare_runtimes():
    ns = list(range(10, 501, 10))
    times_astar = []
    times_dp = []

    for N in ns:
        p = TransportationProblem(N)

        start_time = time.time()
        solutionAstar(p)
        elapsed_time_astar = time.time() - start_time
        times_astar.append(elapsed_time_astar)

        start_time = time.time()
        solutionDP(p)
        elapsed_time_dp = time.time() - start_time
        times_dp.append(elapsed_time_dp)

    # Plotando o gráfico
    plt.plot(ns, times_astar, label='A*')
    plt.plot(ns, times_dp, label='Dynamic Programming')
    plt.xlabel('N')
    plt.ylabel('Tempo de Execução (s)')
    plt.legend()
    plt.show()

# Chamando a função para comparar os tempos de execução
compare_runtimes()

KeyError: ignored

# Exercício 2:

Qual é o caminho mais rápido para ir de 1 a N passando obrigatoriamente por pelo menos 3 números ímpares, novamente sabendo que as duas únicas ações possíveis são:
 - andar de i para i + 1, com duração de 1 minuto (para todo i de 1 a N - 1)
 - mágica de i para 2*i, com duração de 2 minutos (para todo i de 1 a N/2)

Implemente uma nova classe *RestrictedTransportationProblem* para este problema e aplique os algoritmos propostos nas funções *solution*, *solutionAstar* e *solutionDP* para resolvê-lo.

In [ ]:
class RestrictedTransportationProblem:
    def __init__(self, N):
        self.N = N

    def startState(self):
        return 1

    def isEnd(self, state):
        return state == self.N

    def succAndCost(self, state):
        successors = []

        # Adiciona ação 'walk' para estados de 1 a N-1
        if state < self.N:
            successors.append(('walk', state + 1, 1))

        # Adiciona ação 'magic' para estados de 1 a N/2
        if state * 2 <= self.N:
            successors.append(('magic', state * 2, 2))

        return successors

class RestrictedTransportationProblemWithOddCount(RestrictedTransportationProblem):
    def __init__(self, N, min_odd_count=3):
        super().__init__(N)
        self.min_odd_count = min_odd_count

    def startState(self):
        return (1, 0)  # O segundo elemento representa a contagem de números ímpares

    def isEnd(self, state):
        return state[0] == self.N and state[1] >= self.min_odd_count

    def succAndCost(self, state):
        current_position, odd_count = state
        successors = super().succAndCost(current_position)

        for i in range(len(successors)):
            action, next_position, cost = successors[i]
            successors[i] = (action, (next_position, odd_count + next_position % 2), cost)

        return successors


In [ ]:
# Exemplo de teste com diferentes valores de N
for N in [10, 20, 30, 40, 50]:
    # Criar instância do problema restrito com contagem mínima de ímpares como 3
    restricted_problem = RestrictedTransportationProblemWithOddCount(N, min_odd_count=3)

    # Chamar as funções de solução
    print(f"\nN = {N}")
    print("Solution:")
    print(*solution(restricted_problem), sep='\n')

    print("\nA* Solution:")
    print(*solutionAstar(restricted_problem), sep='\n')

    print("\nDP Solution:")
    print(*solutionDP(restricted_problem), sep='\n')



N = 10
Solution:
('walk', (2, 0), 1)
('walk', (3, 1), 1)
('magic', (6, 1), 2)
('walk', (7, 2), 1)
('walk', (8, 2), 1)
('walk', (9, 3), 1)
('walk', (10, 3), 1)

A* Solution:


KeyError: ignored

# Exercício 3:

As matrizes a seguir representam exemplos de mapas de rios congelados que você deve atravessar:

```
m0 =
 [
    "SFFF",
    "FHFH",
    "FFFH",
    "HFFG"
 ]

m1 =
 [
    "SFFFFFFF",
    "FFFFFFFF",
    "FFFHFFFF",
    "FFFFFHFF",
    "FFFHFFFF",
    "FHHFFFHF",
    "FHFFHFHF",
    "FFFHFFFG",
 ]
```

Nesses mapas, *S* representa a sua posição inicial; *F* significa que a posição está congelada e pode ser ocupada; *H* significa que existe um buraco nessa posição; *G* representa a posição onde você quer chegar.

Implemente a classe *FrozenLakeProblem* que recebe um mapa e implementa a mesma interface dos problemas anteriores. As ações permitidas em cada posição são: norte, sul, leste e oeste. Em sequência, encontre o algoritmo que retorna o caminho mais curto para um dado mapa.


In [ ]:
import random

class FrozenLakeProblem:
    def __init__(self, m):
        self.map = m

    def startState(self):
        for i in range(len(self.map)):
            for j in range(len(self.map[i])):
                if self.map[i][j] == 'S':
                    return (i, j)

    def isEnd(self, state):
        i, j = state
        return self.map[i][j] == 'G'

    def succAndCost(self, state):
        i, j = state
        successors = []

        # Ações possíveis: Norte, Sul, Leste, Oeste
        actions = [(i-1, j), (i+1, j), (i, j-1), (i, j+1)]

        for new_i, new_j in actions:
            if 0 <= new_i < len(self.map) and 0 <= new_j < len(self.map[0]) and self.map[new_i][new_j] != 'H':
                cost = 0 if self.map[new_i][new_j] == 'G' else 1
                successors.append(('move', (new_i, new_j), cost))

        return successors


In [ ]:
def solution(problem):
    # Implementação de busca em largura para encontrar o caminho mais curto
    start = problem.startState()
    frontier = [(start, [])]
    visited = set()

    while frontier:
        current_state, path = frontier.pop(0)

        if problem.isEnd(current_state):
            return path

        if current_state not in visited:
            visited.add(current_state)
            for action, next_state, _ in problem.succAndCost(current_state):
                frontier.append((next_state, path + [action]))

# Exemplo de uso:
m0 = [
    "SFFF",
    "FHFH",
    "FFFH",
    "HFFG"
]

problem = FrozenLakeProblem(m0)
print(solution(problem))

['move', 'move', 'move', 'move', 'move', 'move']


Imagine agora que, dado que o lago está congelado, quando você tenta se movimentar nele, existe uma chance de você escorregar e se movimentar em uma direção aleatória perpendicular à direção que você pretendia. Após algumas tentativas, você percebe que você escorrega com probabilidade *p* toda vez que tenta se movimentar e que, quando escorrega, é igual a chance de ir para qualquer um dos dois lados possíveis. Você também percebe que quando escorrega na direção das paredas que limitam o mapa, você bate na parede e permanece na mesma posição. Por último, é importante ressaltar que os buracos também são estados terminais desse MDP, porém com consequência bem mais desagradáveis que o estado G.

Implemente a classe *FrozenLakeMDP* que recebe um mapa e uma probabilidade e implementa as funções necessárias para definir a interface desse problema para utilização com os algoritmos de *policyEvaluation* e *valueIteration* apresentados em sala de aula. Em seguida, implemente os algoritmos de *policyEvaluation* e *valueIteration* nas funções com os respectivos nomes.

In [ ]:
class FrozenLakeMDP:
    def __init__(self, m, p):
        self.map = m
        self.probability = p

    def startState(self):
        for i in range(len(self.map)):
            for j in range(len(self.map[i])):
                if self.map[i][j] == 'S':
                    return (i, j)

    def isEnd(self, state):
        i, j = state
        return self.map[i][j] == 'G' or self.map[i][j] == 'H'

    def states(self):
        return [(i, j) for i in range(len(self.map)) for j in range(len(self.map[0]))]

    def actions(self, state):
        return ['move']

    def succProbReward(self, state, action):
        i, j = state
        successors = []

        # Ações possíveis: Norte, Sul, Leste, Oeste
        actions = [(i-1, j), (i+1, j), (i, j-1), (i, j+1)]

        for new_i, new_j in actions:
            if 0 <= new_i < len(self.map) and 0 <= new_j < len(self.map[0]) and self.map[new_i][new_j] != 'H':
                if self.map[new_i][new_j] == 'G':
                    successors.append(((new_i, new_j), 1 - self.probability, 0))
                else:
                    successors.append(((new_i, new_j), self.probability, 0))

        return successors

In [ ]:
def policyEvaluation(mdp, policy, gamma=0.9, tolerance=1e-6, max_iterations=1000):
    # Implementação do algoritmo de avaliação de política
    V = {state: 0 for state in mdp.states()}

    for _ in range(max_iterations):
        delta = 0
        for state in mdp.states():
            if not mdp.isEnd(state):
                v = V[state]
                new_v = sum(prob * (reward + gamma * V[next_state]) for next_state, prob, reward in mdp.succProbReward(state, policy[state]))
                V[state] = new_v
                delta = max(delta, abs(v - new_v))

        if delta < tolerance:
            break

    return V

def valueIteration(mdp, gamma=0.9, tolerance=1e-6, max_iterations=1000):
    # Implementação do algoritmo de iteração de valor
    V = {state: 0 for state in mdp.states()}

    for _ in range(max_iterations):
        delta = 0
        for state in mdp.states():
            if not mdp.isEnd(state):
                v = V[state]
                new_v = max(sum(prob * (reward + gamma * V[next_state]) for next_state, prob, reward in mdp.succProbReward(state, action)) for action in mdp.actions(state))
                V[state] = new_v
                delta = max(delta, abs(v - new_v))

        if delta < tolerance:
            break

    return V


Por último, execute o algoritmo de *valueIteration* para uma mdp com *p = 0*. Observe o caminho tomado executando a política ótima obtida. Qual a relação desse caminho com o caminho mais curto obtido no item anterior?

## Resposta:

In [ ]:
# Exemplo de uso para valueIteration com p=0:
m0 = [
    "SFFF",
    "FHFH",
    "FFFH",
    "HFFG"
]

mdp = FrozenLakeMDP(m0, p=0)
optimal_values = valueIteration(mdp)
print("Optimal Values:")
print(optimal_values)

# Encontrando a política ótima para o exemplo m0:
optimal_policy = {state: max(mdp.actions(state), key=lambda action: sum(prob * (reward + 0.9 * optimal_values[next_state]) for next_state, prob, reward in mdp.succProbReward(state, action))) for state in mdp.states()}
print("Optimal Policy:")
print(optimal_policy)

Optimal Values:
{(0, 0): 0.0, (0, 1): 0.0, (0, 2): 0.0, (0, 3): 0.0, (1, 0): 0.0, (1, 1): 0, (1, 2): 0.0, (1, 3): 0, (2, 0): 0.0, (2, 1): 0.0, (2, 2): 0.0, (2, 3): 0, (3, 0): 0, (3, 1): 0.0, (3, 2): 0.0, (3, 3): 0}
Optimal Policy:
{(0, 0): 'move', (0, 1): 'move', (0, 2): 'move', (0, 3): 'move', (1, 0): 'move', (1, 1): 'move', (1, 2): 'move', (1, 3): 'move', (2, 0): 'move', (2, 1): 'move', (2, 2): 'move', (2, 3): 'move', (3, 0): 'move', (3, 1): 'move', (3, 2): 'move', (3, 3): 'move'}
